# Midterm

## FINM 25000 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

# Instructions

## Please note the following:

Submission
* You will upload your solution to the `Midterm` assignment on Canvas, where you downloaded this. 
* Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers,)
* and it should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation.
* If you prompt AI for help, you must cite the AI and the prompt. If you use AI embedded coding tools, cite that you used such tools.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 45     |
| 2       | 40     |
| 3       | 35     |


Each numbered question is worth `5 points` unless otherwise specified.

For every minute late you submit the exam, you will lose `1 point`.

***

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_1_data.xlsx`

This file has sheets for...
* `excess returns` - excess returns of some of the biggest companies in the S&P, along with the SPY ETF.

Note the data is **weekly**, so annualization should use a factor of `52`.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

In [2]:
DATAFILE = '../data/midterm_data.xlsx'
SHEET_INFO = 'info'

In [3]:
info = pd.read_excel(DATAFILE,sheet_name=SHEET_INFO)
info.set_index('ticker',inplace=True)

In [4]:
rets = pd.read_excel(DATAFILE,sheet_name='excess returns')
rets.set_index('date',inplace=True)
rets.columns = [s.split(' ')[0] for s in rets.columns]
rets = rets[info.index]

In [5]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,63670226,6.035170e+11,The trust seeks to achieve its investment obje...
AAPL,Apple Inc.,EQUITY,USD,39765812,NaN,"Apple Inc. designs, manufactures, and markets ..."
NVDA,NVIDIA Corporation,EQUITY,USD,193633263,NaN,"NVIDIA Corporation, a computing infrastructure..."
MSFT,Microsoft Corporation,EQUITY,USD,16459512,NaN,Microsoft Corporation develops and supports so...
GOOGL,Alphabet Inc.,EQUITY,USD,34282922,NaN,Alphabet Inc. offers various products and plat...
AMZN,"Amazon.com, Inc.",EQUITY,USD,50518307,NaN,"Amazon.com, Inc. engages in the retail sale of..."
META,"Meta Platforms, Inc.",EQUITY,USD,10873880,NaN,"Meta Platforms, Inc. engages in the developmen..."
TSLA,"Tesla, Inc.",EQUITY,USD,79236442,NaN,"Tesla, Inc. designs, develops, manufactures, l..."
AVGO,Broadcom Inc.,EQUITY,USD,14274674,NaN,"Broadcom Inc. designs, develops, and supplies ..."


In [6]:
rets

,SPY,AAPL,NVDA,MSFT,GOOGL,AMZN,META,TSLA,AVGO,BRK-B,LLY
date,,,,,,,,,,,
2015-01-09,-0.005925,0.024332,-0.009620,0.009014,-0.054624,-0.037748,-0.009232,-0.057862,0.047875,0.001830,-0.002037
2015-01-16,-0.013009,-0.053927,0.000822,-0.020313,0.019271,-0.021028,-0.033112,-0.065942,-0.010477,-0.001921,0.010544
2015-01-23,0.016565,0.065950,0.037575,0.020329,0.061689,0.074465,0.035249,0.042575,0.030437,-0.000603,0.020515
2015-01-30,-0.026840,0.037088,-0.072821,-0.143614,-0.008028,0.134986,-0.024578,0.011567,-0.038234,-0.034848,-0.001712
2015-02-06,0.030493,0.015017,0.062409,0.049662,-0.006918,0.055617,-0.019061,0.067493,0.017989,0.043478,-0.022868
...,...,...,...,...,...,...,...,...,...,...,...
2025-05-30,0.017208,0.028032,0.028705,0.022070,0.018866,0.019457,0.032037,0.020438,0.057825,0.000450,0.033027
2025-06-06,0.019350,0.018092,0.051575,0.024573,0.014103,0.044561,0.080368,-0.145320,0.022884,-0.017889,0.046472
2025-06-13,-0.004571,-0.037631,0.000765,0.008738,0.004701,-0.007882,-0.022268,0.101224,0.006169,-0.013136,0.063271


***

# 1. Risk Statistics and Decomposition

### 1.1. (10pts)

Display a table with the following metrics for each of the return series.

* mean (annualized)
* volatility (annualized)
* Sharpe ratio (annualized)
* skewness
* kurtosis
* maximum drawdown

### 1.2.

As a standalone investment, which is most attractive? And least? Justify your answer.

### 1.3. (10pts)

For each investment, estimate a regression against `SPY`. Report the 
* alpha (annualized as a mean)
* beta
* info ratio
* r-squared

### 1.4.

Based on this table, which investment seems most attractive relative to holding `SPY`? Justify your answer.

### 1.5.

Suppose you expect `AAPL` to do well relatively, but you want to hedge broad market risk (`SPY`) and A.I. risk (`NVDA`). 

For every $100 in `AAPL`, what should you hold in `SPY` and `NVDA`?

Estimate the regression including an intercept.

### 1.6.

Without estimating anything new, consider the idea of replicating `AAPL` using `SPY`, and `NVDA`. Which regression statistic best indicates if your replication tracks the target well?

### 1.7.

In the ProShares case, did we find the attempts at hedge-fund replication were successful?

Specifically, did we achieve high **correlation** to the...
* Merril Lynch Benchmark?
* HFRI Index?

Were there any drawbacks to using our replication rather than the direct product?


***

# 2. Portfolio Allocation

### 2.1.

Display the correlation matrix of the returns.

Based on this information, which investment do you anticipate will get extra weight in the portfolio, beyond what it would merit for its mean return? Explain.

### 2.2.

Calculate and report the weights of the mean-variance optimized portfolio, also called the tangency portfolio. 

*Note that these are excess returns.*

### 2.3.

Report the following performance statistics of the portfolio achieved with the optimized weights calculated above.
* mean
* volatility
* Sharpe

(Annualize all three statistics.)

### 2.4.

Consider the biggest positive weight (long) and most negative weight (short).

Do they align with the most extreme Sharpe ratios? Explain.

### 2.5.

Try dropping `SPY` from the set of assets. 

Re-run the optimization and report the new tangency weights.

### 2.6.

Mark each of the statements as `True or False`. No justification is needed.

In our analysis of the multi-asset portfolio optimization, we found that a change in TIPS mean excess returns caused a large change in the...

* performance of the tangency portfolio.
* weights of the tangency portfolio.
* correlation structure of the assets.

### 2.7. (10pts)

1. Briefly explain why the optimized portfolio is unrealistic in practice.

1. What does Harvard do to make the optimization more practical.

1. Why did Harvard optimize in levels (securities within an asset class, then all the broad asset classes) rather than direclty optimizing all the securities.

****

# 3. Expected Returns

### 3.1.

Consider the CAPM as tested with a single stock,

$$\mathbb{E}\left[r\right] = \beta\,  \mathbb{E}\left[x\right]$$

where
* $r$ denotes the return on `NVDA`, and ETF for U.S. oil
* $x$ denotes the returns on `SPY`, an ETF for USD currency trades.

Estimate the associated regression,
$$r_t = \alpha + \beta x_t + \epsilon_t$$

Report,
* $\alpha$
* $\beta$
* r-squared

Annualize alpha.

### 3.2.

What evidence is there that this (simplistic) factor pricing model does not price `NVDA` correctly?

### 3.3.

What is the (annualized) average excess return of `NVDA`?

According to the CAPM, what level of average excess return is explained by the factor risk?

### 3.4.

Now let's disregard the equilibrium pricing model above, and try calculating the expected return of `NVDA` via forecasting signals. To keep things simple, just use a forecasting regression (rather than a neural network, regression tree, etc.)
$$r_{t+1} = \alpha + \beta x_t + \epsilon_{t+1}$$

We estimate a forecasting regression of `NVDA` on `SPY`.


From this **forecasting** regression, report
* $\alpha$
* $\beta$
* r-squared

### 3.5.

Given the stats in `3.4.`, what do you think of this forecast for `NVDA`? Be specific.

### 3.6.

According to the signal forecasting model, what is the expected return (annualized) of `NVDA` in `July 2025`?

### 3.7.

Why is it important to create style factors which go **long** and **short** the targeted style?

****